In [1]:
!pip install pypdf

In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [3]:
## Embedding
!pip install install sentence_transformers

In [4]:
!pip install llama_index==0.9.40


In [5]:
from llama_index import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [6]:
documents=SimpleDirectoryReader("/content/Data").load_data()
documents

[Document(id_='a449d30b-58c1-4e7d-984b-d922c60d2686', embedding=None, metadata={'page_label': '1', 'file_name': 'armstrong_neil.pdf', 'file_path': '/content/Data/armstrong_neil.pdf', 'file_type': 'application/pdf', 'file_size': 51425, 'creation_date': '2024-04-01', 'last_modified_date': '2024-04-01', 'last_accessed_date': '2024-04-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="   \n \n \n \n \nNEIL A. ARMSTRONG   \nNASA ASTRONAUT ( DECEASED ) \n PERSONAL DATA:   Born  on August 5, 1930,  in Wapakoneta, Ohio .  Married.  Two sons.  \nDied on August  25, 2012.  \n \nEDUCATION:   Armstrong  received a Bachelor of Science in Aeronautical Engineering from \nPurdue University and a Master of Science in Aerospace Engineering from the University of \nSout

In [7]:
system_prompt="""
You are an AI assistant. Your goal is to Summarize the lesson plans for the teacher, which has been provided to you.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [9]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [11]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [12]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7af6d266a410>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7af6d266a410>, id_func=<function default_id_func at 0x7af7a89dc0d0>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.logger.base.LlamaLogger object at 0x7af6dd1f1d80>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7af6d266a410>)

In [13]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [14]:
index

In [15]:
query_engine=index.as_query_engine()

In [16]:
response=query_engine.query("Provide me workflow to explain kids entire biographical data of Neil Armstrong")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [17]:
print(response)



Workflow:

Step 1: Introduction

* Introduce Neil Armstrong as a famous astronaut and engineer.
* Explain that he was born in Ohio and became the first person to walk on the moon.

Step 2: Personal Data

* Share Armstrong's birthdate and hometown.
* Explain that he married and had two sons.

Step 3: Education

* Discuss Armstrong's educational background, including his degree in aeronautical engineering from Purdue University and his master's degree in aerospace engineering from the University of Southern California.
* Highlight his honorary doctorates from multiple universities.

Step 4: Special Honors

* List Armstrong's special honors, including his fellowships in the Society of Experimental Test Pilots and the Royal Aeronautical Society.
* Explain that he was also an honorary fellow of the American Institute of Aeronautics and Astronautics and the International Astronautics Federation.

Step 5: Experience

* Describe Armstrong's experience as a naval aviator and his service in th